In [2]:
import torch
torch.cuda.is_available()

True

In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import torch

from transformations.transform import WaveformToInput as TorchTransform
from architecture.backend.yamnet.params import YAMNetParams
from architecture.backend.yamnet.model import yamnet
from architecture.backend.yamnet.model import yamnet_category_metadata

from architecture.classifier.classification import Classifier

from datasets.SvdExDataset import SvdCutOffShort
%load_ext autoreload
%autoreload 2

In [3]:
from tqdm import tqdm
from torch.utils.data import DataLoader
import math

dataset = SvdCutOffShort(r"/Users/yiftachedelstain/Development/VoiceDisorderIdentification/data",classification_binary=True)
loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2
)
def train_val_test_split(ds):
        ds_len =  len(ds)
        len_test = math.floor(ds_len*0.2)
        len_train_vald = ds_len - len_test
        len_train = math.ceil(len_train_vald*0.8)
        len_vald = len_train_vald - len_train
        return torch.utils.data.random_split(ds, [len_train,len_vald,len_test]) 

train,val,test = train_val_test_split(dataset)
# print(len(train),len(val),len(test),len(dataset))
pt_model = Classifier([512,64])
# print(pt_model)
pt_model.eval()
print(len(dataset))
with torch.no_grad():

# print(pt_model)
    for epoch in range(2):
        with tqdm(loader) as t:
            for idx,x in enumerate(t):
                # x = torch.from_numpy(patches)
                # x = x.unsqueeze(1)  # [5, 96, 64] -> [5, 1, 96, 64]
                t.set_description(f"epoch {epoch}: train loss is {idx}")
                pt_pred = pt_model(x['data'])
                # print(pt_pred['classification'])
                # print(x['classification'])
                # assert(x['classification'].all())
                # print(pt_pred.shape)

AssertionError: Directory should not be empty

In [ ]:
import trainer.train_svd 
import torch.nn as nn
import torch.optim 
dataset = SvdCutOffShort(r"/home/yiftach.ede@staff.technion.ac.il/Desktop/SVD",classification_binary=True,overfit_test = False)
model = Classifier([512,128])
loss = nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(model.parameters())
hyper_params = {
    'train_batch_size':128,
    'vald_batch_size':128,
    'test_batch_size':128,
    'num_workers':5,
    'epochs':100
}
trainer = trainer.train_svd.Trainer(dataset=dataset,model=model,optimizers=opt,critereon=loss,hyper_params=hyper_params)
model = trainer.train()
trainer.test(model[0])